## Dependencies

In [1]:
! pip install -U grape

In [2]:
! pip freeze | grep "grape"
! pip freeze | grep "embiggen"
! pip freeze | grep "ensmallen"

grape==0.1.24
embiggen==0.11.38
ensmallen==0.8.24


In [31]:
!pip install pykeen

     |████████████████████████████████| 717 kB 5.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 348 kB 70.4 MB/s eta 0:00:01
     |█████████████████████████████▋  | 824.0 MB 75.5 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████████▉| 884.5 MB 84.7 MB/s eta 0:00:01

     |████████████████████████████████| 888.8 MB 84.7 MB/s eta 0:00:01

     |████████████████████████████████| 890.2 MB 84.7 MB/s eta 0:00:01

     |████████████████████████████████| 890.2 MB 10 kB/s 
     |████████████████████████████████| 209 kB 71.9 MB/s eta 0:00:01
     |████████████████████████████████| 80 kB 1.2 MB/s  eta 0:00:01
     |████████████████████████████████| 78 kB 782 kB/s  eta 0:00:01
     |████████████████████████████▍   | 495.1 MB 83.3 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 556.8 MB 85.3 MB/s eta 0:00:01

     |████████████████████████████████| 557.1 MB 12 kB/s s eta 0:00:01
     |████████████████████████████████| 317.1 MB 48 kB/s s eta 0:00:01
     |████████████████████████████████| 849 kB 75.9 MB/s eta 0:00:01
     |████████████████████████████████| 21.0 MB 55.9 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 69.4 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 6.4 MB/s  eta 0:00:01
     |████████████████████████████████| 112 kB 72.8 MB/s eta 0:00:01
  Created wheel for click-default-group: filename=click_default_group-1.2.2-py3-none-any.whl size=3385 sha256=968ed21752177016c08165f7e2d8b106d66247efc25cd560d3830caaa88cf4c0
  Stored in directory: /home/ubuntu/.cache/pip/wheels/f1/e3/67/f79591b232f0deb8e51317fa288eef8b996e2d6cf033bdc1a0
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=fe14bdd58431ccbc450997a25344a5ed10f66ccb9d21a79a95e63c8520182af5
  Stored in directory: /home/ubuntu/.cache/pip/wheels/0c/09/9e/49e21

In [ ]:
# imports
from grape import Graph
import pandas as pd

from grape.edge_prediction import PerceptronEdgePrediction
from grape.embedders import FirstOrderLINEEnsmallen
# from grape.datasets.string import HomoSapiens

2022-11-02 05:22:14.992226: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-02 05:22:14.992278: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
!ls ./input
# ! mv ./input/merged_edge_list\ \(1\).tsv ./input/merged_edge_list.tsv
# ! mv ./input/merged_node_list\ \(1\).tsv ./input/merged_node_list.tsv

## Independent evaluation of different edges

In [ ]:
g = Graph.from_csv(
  directed=False, 
  node_path='./input/merged_node_list.tsv',
  edge_path='./input/merged_edge_list.tsv',
  verbose=True,
  nodes_column='node',
  node_list_node_types_column='node_type',
  default_node_type='None',
  sources_column='head',
  destinations_column='tail',
  edge_list_edge_types_column='relation',
  name="Mito KG"
)
g = g.remove_disconnected_nodes()

In [ ]:
# how many of each edge type?
g.get_edge_type_names_counts_hashmap()

In [ ]:
def label_predictions_with_ground_truth(pred_df, test_graph, return_bool = True, directed=False):
    test_edges = set(test_graph.get_edge_node_names(directed=False))
    # consider reverse edges if directed is False
    if not directed:
        test_edges = test_edges.union([(t,f) for (f,t) in test_edges])
    
    # convert to dataframe to label edges
    test_edges_df = pd.DataFrame(test_edges)
    test_edges_df.columns = ['sources','destinations']
    
    labeled_pred_df = pred_df.merge(test_edges_df,how='left',indicator=True)
    test_truth = []
    for b in labeled_pred_df['_merge'] == 'both':
        if return_bool:
            y_ = b
        else:
            # return 1 or 0
            y_ = 1 if b else 0
        test_truth += [y_]
    labeled_pred_df['ground_truth'] = test_truth
    labeled_pred_df = labeled_pred_df.drop('_merge', axis=1)
    return labeled_pred_df

# def label_predictions_with_ground_truth(pred_df, test_graph, return_bool = True):
#     test_edges = pd.DataFrame(test_graph.get_edge_node_names(directed=False))
#     test_edges.columns = ['sources','destinations']
    
#     labeled_pred_df = pred_df.merge(test_edges,how='left',indicator=True)
#     test_truth = []
#     for b in labeled_pred_df['_merge'] == 'both':
#         if return_bool:
#             y_ = b
#         else:
#             # return 1 or 0
#             y_ = 1 if b else 0
#         test_truth += [y_]
#     labeled_pred_df['ground_truth'] = test_truth
#     labeled_pred_df = labeled_pred_df.drop('_merge', axis=1)
#     return labeled_pred_df


def label_negative_sample_pred(pred_df):
    pred_df['ground_truth'] = False
    return pred_df


def evaluate_predictions(m, labeled_pred_df):
    # convert DataFrame to numpy used for evaluation
    y_label = labeled_pred_df['ground_truth'].to_numpy()
    y_score = labeled_pred_df['predictions'].to_numpy()
    
    # handle case where no ground truth (for negative sampled edges)
    all_false = len(set(y_label)) == 1 and (~y_label[0])
    if all_false:
        e1 = {'auroc':float('NaN'),'auprc':float('NaN')}
    else:
        e1 = m.evaluate_prediction_probabilities(y_label,y_score)
    e2 = m.evaluate_predictions(y_label,y_score)
    return e1 | e2

In [ ]:
def get_edge_type_to_node_types_mapping(g, directed=False):
    '''
    This function returns a mapping from edge_type -> (from_types,to_types)
    '''
    # gather node and edge types for every edge as a DataFrame
    edge_to_type_dict = {h:[] for h in ["from","to","from_type","to_type","edge_type"]}
    for from_node_id,to_node_id in g.get_edge_node_ids(directed=False):
        # get node types
        from_node_type_ids = g.get_node_type_ids_from_node_id(from_node_id)
        from_node_type = [g.get_node_type_name_from_node_type_id(i) for i in from_node_type_ids]
        to_node_type_ids = g.get_node_type_ids_from_node_id(to_node_id)
        to_node_type =  [g.get_node_type_name_from_node_type_id(i) for i in to_node_type_ids]

        # get edge type
        edge_id = g.get_edge_id_from_node_ids(from_node_id,to_node_id)
        edge_type = g.get_edge_type_name_from_edge_id(edge_id)

        # append to dict
        edge_to_type_dict['from'] += [from_node_id]
        edge_to_type_dict['to'] += [to_node_id]
        edge_to_type_dict['from_type'] += [from_node_type]
        edge_to_type_dict['to_type'] += [to_node_type]
        edge_to_type_dict['edge_type'] += [edge_type]
    edge_to_type_df = pd.DataFrame(edge_to_type_dict)

    # take unique node types for each edge type
    edge_type_to_node_types = {}
    for edge_type in set(edge_to_type_df['edge_type']):
        # only rows with the specified edge_type
        sub_df = edge_to_type_df[edge_to_type_df['edge_type'] == edge_type]
        
        # get unique pairs of node types
        unique_node_type_pairs = set()
        for from_node_types, to_node_types in zip(sub_df['from_type'],sub_df['to_type']):
            # enumerate all pairs, since these are lists of node types
            pairs = [(f,t) for f in from_node_types for t in to_node_types]
            
            # if undirected, do not include reverse node type 
            # i.e. include (type_1,type_2) but not (type_2,type_1)
            if not directed:                
                pairs_sorted = set()
                for f,t in pairs:
                    # sort pair alphabetically
                    pair = (f,t) if (f < t) else (t,f) 
                    # keep unique pairs
                    pairs_sorted.add(pair)
                pairs = pairs_sorted
                
            # add unique pairs
            unique_node_type_pairs = unique_node_type_pairs.union(pairs)
            
        edge_type_to_node_types[edge_type] = list(unique_node_type_pairs)
    return edge_type_to_node_types

edge_type_to_node_types_mapping = get_edge_type_to_node_types_mapping(g,directed=False)
edge_type_to_node_types_mapping

In [ ]:
def independent_edge_evaluation(g, node_types):
    
    # get the node types we will be predicting edge between
    source_node_type_list = [node_types[0]]
    destination_node_type_list = [node_types[1]]
    print(source_node_type_list,destination_node_type_list)
    
    # split graph into train/test
    train, test = g.connected_holdout(train_size=0.7)

    # train embedding on train graph
    embedding = FirstOrderLINEEnsmallen().fit_transform(train)

    # train model on train graph
    model = PerceptronEdgePrediction(
        edge_features=None,
        number_of_edges_per_mini_batch=32,
        edge_embeddings="CosineSimilarity"
    )
    model.fit(
        graph=train, 
        node_features=embedding
    )
    
    # predictions for train, test, and negative sampled graph
    train_pred_df = model.predict_proba_bipartite_graph_from_edge_node_types(graph=train, 
                                        node_features=embedding, 
                                        source_node_types = source_node_type_list,
                                        destination_node_types = destination_node_type_list,
                                        return_predictions_dataframe=True
                                    )
    test_pred_df = model.predict_proba_bipartite_graph_from_edge_node_types(graph=test, 
                                        node_features=embedding, 
                                        source_node_types = source_node_type_list,
                                        destination_node_types = destination_node_type_list,
                                        return_predictions_dataframe=True
                                    )
    negative_sampled_pred_df = model.predict_proba_bipartite_graph_from_edge_node_types(
                                    graph=g.sample_negative_graph(number_of_negative_samples=test.get_number_of_edges()),
                                    node_features=embedding, 
                                    source_node_types = source_node_type_list,
                                    destination_node_types = destination_node_type_list,
                                    return_predictions_dataframe=True
                                )
    
    # label predictions
    labeled_train_pred_df = label_predictions_with_ground_truth(train_pred_df,train)
    labeled_test_pred_df = label_predictions_with_ground_truth(test_pred_df,test)
    labeled_negative_sample_pred_df = label_negative_sample_pred(negative_sampled_pred_df)
    
    # evaluation
    data = [evaluate_predictions(model,labeled_train_pred_df),
            evaluate_predictions(model,labeled_test_pred_df),
            evaluate_predictions(model,labeled_negative_sample_pred_df)]
    eval_df = pd.DataFrame(data)
    
    return eval_df

edge_pair = edge_type_to_node_types_mapping['CaseOLAP_score'][0]
independent_edge_evaluation(g,edge_pair)

In [ ]:
import os
import yaml
import pandas as pd
from grape import Graph

# node embedding imports
from embiggen.embedders.ensmallen_embedders.degree_spine import DegreeSPINE
from embiggen.embedders import GLEEEnsmallen
from embiggen.embedders import HOPEEnsmallen
from embiggen.embedders.pykeen_embedders.distmult import DistMultPyKEEN
from embiggen.embedders.pykeen_embedders.hole import HolEPyKEEN

# edge prediction imports
from embiggen.edge_prediction.edge_prediction_model import AbstractEdgePredictionModel
from grape.edge_prediction import PerceptronEdgePrediction
from grape.edge_prediction import MLPEdgePrediction
from grape.edge_prediction import GNNEdgePrediction


def label_predictions_with_ground_truth(pred_df, test_graph, return_bool = True):
    test_edges = pd.DataFrame(test_graph.get_edge_node_names(directed=False))
    test_edges.columns = ['sources','destinations']
    
    labeled_pred_df = pred_df.merge(test_edges,how='left',indicator=True)
    test_truth = []
    for b in labeled_pred_df['_merge'] == 'both':
        if return_bool:
            y_ = b
        else:
            # return 1 or 0
            y_ = 1 if b else 0
        test_truth += [y_]
    labeled_pred_df['ground_truth'] = test_truth
    labeled_pred_df = labeled_pred_df.drop('_merge', axis=1)
    return labeled_pred_df


def label_negative_sample_pred(pred_df):
    pred_df['ground_truth'] = False
    return pred_df


def evaluate_predictions(m, labeled_pred_df):
    # convert DataFrame to numpy used for evaluation
    y_label = labeled_pred_df['ground_truth'].to_numpy()
    y_score = labeled_pred_df['predictions'].to_numpy()
    
    # handle case where no ground truth (for negative sampled edges)
    all_false = len(set(y_label)) == 1 and (~y_label[0])
    if all_false:
        e1 = {'auroc':float('NaN'),'auprc':float('NaN')}
    else:
        e1 = m.evaluate_prediction_probabilities(y_label,y_score)
    e2 = m.evaluate_predictions(y_label,y_score)
    return e1 | e2

def get_edge_type_to_node_types_mapping(g, directed=False):
    '''
    This function returns a mapping from edge_type -> (from_types,to_types)
    '''
    # gather node and edge types for every edge as a DataFrame
    edge_to_type_dict = {h:[] for h in ["from","to","from_type","to_type","edge_type"]}
    for from_node_id,to_node_id in g.get_edge_node_ids(directed=False):
        # get node types
        from_node_type_ids = g.get_node_type_ids_from_node_id(from_node_id)
        from_node_type = [g.get_node_type_name_from_node_type_id(i) for i in from_node_type_ids]
        to_node_type_ids = g.get_node_type_ids_from_node_id(to_node_id)
        to_node_type =  [g.get_node_type_name_from_node_type_id(i) for i in to_node_type_ids]

        # get edge type
        edge_id = g.get_edge_id_from_node_ids(from_node_id,to_node_id)
        edge_type = g.get_edge_type_name_from_edge_id(edge_id)

        # append to dict
        edge_to_type_dict['from'] += [from_node_id]
        edge_to_type_dict['to'] += [to_node_id]
        edge_to_type_dict['from_type'] += [from_node_type]
        edge_to_type_dict['to_type'] += [to_node_type]
        edge_to_type_dict['edge_type'] += [edge_type]
    edge_to_type_df = pd.DataFrame(edge_to_type_dict)

    # take unique node types for each edge type
    edge_type_to_node_types = {}
    for edge_type in set(edge_to_type_df['edge_type']):
        # only rows with the specified edge_type
        sub_df = edge_to_type_df[edge_to_type_df['edge_type'] == edge_type]
        
        # get unique pairs of node types
        unique_node_type_pairs = set()
        for from_node_types, to_node_types in zip(sub_df['from_type'],sub_df['to_type']):
            # enumerate all pairs, since these are lists of node types
            pairs = [(f,t) for f in from_node_types for t in to_node_types]
            
            # if undirected, do not include reverse node type 
            # i.e. include (type_1,type_2) but not (type_2,type_1)
            if not directed:                
                pairs_sorted = set()
                for f,t in pairs:
                    # sort pair alphabetically
                    pair = (f,t) if (f < t) else (t,f) 
                    # keep unique pairs
                    pairs_sorted.add(pair)
                pairs = pairs_sorted
                
            # add unique pairs
            unique_node_type_pairs = unique_node_type_pairs.union(pairs)
            
        edge_type_to_node_types[edge_type] = list(unique_node_type_pairs)
    return edge_type_to_node_types


subgraph_description_file = "./subgraphs/subgraphs_summary.tsv"
results_table_file_name = "./2022-09-20_independent_evaluation_results_table.csv"

# file listing all sub-graphs to test
subgraph_df = pd.read_csv(subgraph_description_file,sep="\t")

for node_path, edge_path, kg_description, kg_num in zip(subgraph_df['Node file'],
                                                        subgraph_df['Edge file'],
                                                        subgraph_df['Resources Included'],
                                                        subgraph_df['Experiment Number']):
    kg_name = 'CKG %d: %s'%(kg_num,kg_description)

    # load graph
    g = Graph.from_csv(
      directed=False,
      node_path=node_path,
      edge_path=edge_path,
      verbose=True,
      nodes_column='id',
      node_list_node_types_column='category',
      default_node_type='None',
      sources_column='subject',
      destinations_column='object',
      edge_list_edge_types_column='predicate',
      weights_column = 'score',
      default_weight=1.0,
      name=kg_name
    )
    g = g.remove_disconnected_nodes()
    
    # split graph into train/test
    train, test = g.connected_holdout(train_size=0.7)

    # load edge types
    edge_type_to_node_types_mapping = get_edge_type_to_node_types_mapping(g,directed=False)
    # to compute edge evaluation only for CaseOLAP_score edges
    edge_pair = edge_type_to_node_types_mapping['CaseOLAP_score'][0]
    
    # train all node embeddings
    embedding_models = {'SPINE':DegreeSPINE().fit_transform(train),
                    'GLEE':GLEEEnsmallen().fit_transform(train),
                    'HOPE':HOPEEnsmallen().fit_transform(train),
                    'DistMultPyKEEN':DistMultPyKEEN().fit_transform(train),
                    'HolEPyKEEN':HolEPyKEEN().fit_transform(train)}
    
    # store results of gridsearch here
    results_df = None

    # grid search over node embedding models 
    for node_embedding_model_name,embedding in node_models.items():
                
        # grid search over edge prediction models
        edge_prediciton_models = {'Perceptron': PerceptronEdgePrediction(edge_features=None,
                                           number_of_edges_per_mini_batch=32,
                                           edge_embeddings="CosineSimilarity"),
                                  'MLP': MLPEdgePrediction(edge_features=None,
                                           number_of_edges_per_mini_batch=32,
                                           edge_embeddings="CosineSimilarity"),
                                  'GNN': GNNEdgePrediction(edge_features=None,
                                           number_of_edges_per_mini_batch=32,
                                           edge_embeddings="CosineSimilarity")}
        
        for edge_pred_model_name, model in edge_prediciton_models.items():
            
            # train model
            model.fit(
                graph=train, 
                node_features=embedding
            )
            
            # predictions for train, test, and negative sampled graph
            train_pred_df = model.predict_proba_bipartite_graph_from_edge_node_types(graph=train, 
                                                node_features=embedding, 
                                                source_node_types = source_node_type_list,
                                                destination_node_types = destination_node_type_list,
                                                return_predictions_dataframe=True
                                            )
            test_pred_df = model.predict_proba_bipartite_graph_from_edge_node_types(graph=test, 
                                                node_features=embedding, 
                                                source_node_types = source_node_type_list,
                                                destination_node_types = destination_node_type_list,
                                                return_predictions_dataframe=True
                                            )
            negative_sampled_pred_df = model.predict_proba_bipartite_graph_from_edge_node_types(
                                            graph=g.sample_negative_graph(number_of_negative_samples=test.get_number_of_edges()),
                                            node_features=embedding, 
                                            source_node_types = source_node_type_list,
                                            destination_node_types = destination_node_type_list,
                                            return_predictions_dataframe=True
                                        )

            # label predictions
            labeled_train_pred_df = label_predictions_with_ground_truth(train_pred_df,train)
            labeled_test_pred_df = label_predictions_with_ground_truth(test_pred_df,test)
            labeled_negative_sample_pred_df = label_negative_sample_pred(negative_sampled_pred_df)

            # evaluation
            data = [evaluate_predictions(model,labeled_train_pred_df),
                    evaluate_predictions(model,labeled_test_pred_df),
                    evaluate_predictions(model,labeled_negative_sample_pred_df)]
            
            
            ### Make result dataframe ###
            eval_df = pd.DataFrame(data)

            # add experiment column
            experiments = ['test','train','negative_sample']
            eval_df.insert(loc=0, column='experiment', value=experiments)
            
            # add edge prediction and node embedding models
            eval_df.insert(loc=0, column='edge_prediction_model', value = edge_pred_model_name)
            eval_df.insert(loc=0, column='node_embedding', value = node_embedding_model_name)
            
            # add kg graph name
            eval_df.insert(loc=0, column='graph_name', value = kg_name)
            
            # store and save results into one big table
            if not results_df:
                results_df = eval_df
            # append new results
            results_df = results_df.append(eval_df, ignore_index = True)
            results_df.to_csv(results_table_file_name)
            
            exp_tag = "kg: %s, node_emb: %s, edge_pred_model: %s"%(kg_name,
                                                                   node_embedding_model_name,
                                                                   ege_pred_model_name)
            print("Finished %s"%exp_tag)
print("Completed.")